In [2]:
import os
from collections import Counter
from itertools import product
from tqdm import tqdm
import torch
import math

In [3]:


folder_path = 'dat410_europarl/'

translations = {}
file_list = os.listdir(folder_path)
text_files = [file for file in file_list]
for text_file in text_files:
    file_path = os.path.join(folder_path, text_file)
    with open(file_path, 'r') as file:
        translations[file_path.split(".")[-1]] = [l[:-1] for l in file.readlines()]
        
translations["en"]

['i declare resumed the session of the european parliament adjourned on friday 17 december 1999 , and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .',
 'although , as you will have seen , the dreaded &apos; millennium bug &apos; failed to materialise , still the people in a number of countries suffered a series of natural disasters that truly were dreadful .',
 'you have requested a debate on this subject in the course of the next few days , during this part-session .',
 'in the meantime , i should like to observe a minute &apos; s silence , as a number of members have requested , on behalf of all the victims concerned , particularly those of the terrible storms , in the various countries of the european union .',
 'please rise , then , for this minute &apos; s silence .',
 'you will be aware from the press and television that there have been a number of bomb explosions and killings in sri lanka .',
 'one of the people assa

# A

In [4]:
swe_count = Counter(" ".join(translations["sv"]).split(" "))
eng_count = Counter(" ".join(translations["en"]).split(" "))
print(swe_count.most_common(20))
print(eng_count.most_common(20))

[('.', 9648), ('att', 9181), (',', 8876), ('och', 7038), ('i', 5949), ('det', 5687), ('som', 5028), ('fÃ¶r', 4959), ('av', 4013), ('Ã¤r', 3840), ('en', 3724), ('vi', 3211), ('jag', 3093), ('den', 2953), ('de', 2930), ('om', 2909), ('till', 2837), ('pÃ¥', 2783), ('har', 2484), ('inte', 2385)]
[('the', 19322), (',', 13514), ('.', 9774), ('of', 9312), ('to', 8801), ('and', 6946), ('in', 6090), ('is', 4400), ('that', 4357), ('a', 4269), ('we', 3223), ('this', 3222), ('i', 2964), ('for', 2942), ('be', 2550), ('it', 2426), ('on', 2304), ('which', 2026), ('are', 1917), ('have', 1881)]


In [5]:
swe_tot = sum(list(swe_count.values()))
swe_word_prob = {w:c/swe_tot for w,c in swe_count.items()}
eng_tot = sum(list(eng_count.values()))
eng_word_prob = {w:c/eng_tot for w,c in eng_count.items()}
sum(list(swe_word_prob.values())), sum(list(eng_word_prob.values()))

(0.9999999999998146, 1.0000000000001097)

Since there are no mentions of zebras in the corpus the probability is estimated to zero. 

In [6]:
eng_word_prob["speaker"], eng_word_prob.get("zebra", 0)


(3.552700763120124e-05, 0)

# B

In [7]:
vocab = [t[0] for t in eng_count.most_common()]
vocab = ["UNK"] + vocab[:int(len(vocab) * 0.30)]
print(sum([eng_count[w] for w in vocab]) / sum(eng_count.values()), len(vocab))

0.948627946965283 3331


In [8]:
def replace_unk(word):
    if not vocab.__contains__(word):
        return "UNK"
    return word

In [9]:
text = " ".join(translations["en"]).split(" ")

bi_count =  Counter([(replace_unk(w1),replace_unk(w2)) for w1,w2 in zip(text[:-1],text[1:])])
bi_count.most_common()

[(('of', 'the'), 2951),
 (('the', 'UNK'), 1795),
 (('in', 'the'), 1664),
 (('UNK', ','), 1266),
 (('.', 'i'), 1186),
 (('UNK', '.'), 1181),
 (('the', 'commission'), 1137),
 (('to', 'the'), 1126),
 (('.', 'the'), 1095),
 (('UNK', 'of'), 1072),
 ((',', 'the'), 971),
 (('UNK', 'UNK'), 956),
 (('it', 'is'), 928),
 (('that', 'the'), 866),
 ((',', 'and'), 863),
 (('the', 'european'), 855),
 (('on', 'the'), 840),
 (('UNK', 'and'), 839),
 (('.', 'we'), 818),
 (('&apos;', 's'), 756),
 ((',', 'i'), 744),
 (('for', 'the'), 730),
 (('and', 'the'), 698),
 (('to', 'be'), 694),
 (('to', 'UNK'), 661),
 (('and', 'UNK'), 646),
 (('UNK', 'the'), 624),
 (('of', 'UNK'), 610),
 (('a', 'UNK'), 608),
 (('.', 'it'), 605),
 (('president', ','), 579),
 (('UNK', 'to'), 566),
 (('.', 'this'), 563),
 ((',', 'in'), 551),
 ((',', 'we'), 547),
 ((',', 'UNK'), 525),
 (('UNK', 'in'), 522),
 (('i', 'would'), 500),
 (('with', 'the'), 500),
 (('.', 'in'), 490),
 (('like', 'to'), 483),
 ((',', 'which'), 475),
 (('this', 'is

Unseen bi gram

In [10]:
w1, w2 = vocab[567], vocab[123]
print(w1,w2)
bi_count[(w1, w2)]

include your


0

In [11]:
bi_total = sum(list(bi_count.values()))
bi_probs = {w: c / bi_total for w,c in bi_count.items()}
bi_probs

{('i', 'declare'): 1.0658140154543033e-05,
 ('declare', 'resumed'): 1.0658140154543033e-05,
 ('resumed', 'the'): 1.0658140154543033e-05,
 ('the', 'session'): 1.0658140154543033e-05,
 ('session', 'of'): 1.0658140154543033e-05,
 ('of', 'the'): 0.010484057198685497,
 ('the', 'european'): 0.003037569944044764,
 ('european', 'parliament'): 0.000770938804511946,
 ('parliament', 'UNK'): 6.39488409272582e-05,
 ('UNK', 'on'): 0.0006892263966604494,
 ('on', 'friday'): 2.486899369393374e-05,
 ('friday', '17'): 3.5527133848476773e-06,
 ('17', 'december'): 7.1054267696953545e-06,
 ('december', '1999'): 2.486899369393374e-05,
 ('1999', ','): 9.592326139088728e-05,
 (',', 'and'): 0.003065991651123546,
 ('and', 'i'): 0.000859756639133138,
 ('i', 'would'): 0.0017763566924238387,
 ('would', 'like'): 0.0011084465760724754,
 ('like', 'once'): 2.486899369393374e-05,
 ('once', 'again'): 0.00024158451016964206,
 ('again', 'to'): 7.815969446664891e-05,
 ('to', 'wish'): 3.5527133848476773e-06,
 ('wish', 'you')

All contains unseen bi gram (include, your) yet they all have non 0 probabilities

In [12]:
def calc_prob(text):
    words = text.split(" ")
    pairs = list(zip(words[:-1], words[1:]))
    log_sum = 0
    for pair in pairs:
        prob = bi_probs.get(pair, eng_word_prob[pair[0]] * eng_word_prob[pair[1]])
        log_prob = math.log(prob)
        log_sum += log_prob 

    return math.exp(log_sum) ** (1/len(pairs))
print(calc_prob("a petition has already been presented to include your petition for a"))
print(calc_prob("a petition has already been presented to include your petition for a large"))
print(calc_prob("a petition has already been presented to include your petition for a dog"))
print(calc_prob("has already"))

2.830739864856002e-06
3.808185963290893e-06
2.0348321091454016e-06
0.0001421085353939071


# C

In [13]:
swe_sentences = translations["sv"]
eng_sentences = translations["en"]

In [14]:
os.path.exists("translator.pth")

True

In [15]:
from random import random
epochs = 10

if not os.path.exists("translator.pth"):
    translator = {}
    for t in tqdm(range(epochs)):
        count_swe_eng = {}
        count_eng = {}
        for eng_sent,swe_sent in (zip([e.split(" ") for e in eng_sentences], [s.split(" ") for s in swe_sentences])):
            for swe_word in swe_sent:
                denominator = sum([translator.get((swe_word, eng_word), random()) for eng_word in eng_sent])
                for eng_word in eng_sent:
                    delta = translator.get((swe_word, eng_word), random()) / denominator
                    count_swe_eng[(eng_word, swe_word)] = count_swe_eng.get((eng_word, swe_word), 0) + delta
                    count_eng[eng_word] = count_eng.get(eng_word, 0 ) + delta
        
        for (eng_word, swe_word), count in count_swe_eng.items():
            translator[(swe_word,eng_word)] = count / count_eng[eng_word]
    torch.save(translator, "translator.pth")
translator = torch.load("translator.pth")

In [16]:
translator[("jag", "i")]

0.9429903644926063

In [17]:
translator_dict = {}
for (swe_word, eng_word), prob in translator.items():
    eng_dict = translator_dict.get(eng_word, {})
    eng_dict[swe_word] = prob
    translator_dict[eng_word] = eng_dict


In [18]:

sorted(translator_dict["european"], key=lambda k: translator_dict["european"][k], reverse=True)[:10]

['europeiska', 'europeisk', 'i', 'den', 'en', '.', ',', 'till', 'att', 'och']

In [26]:
sentence = "mrs lynne , you are quite right and i shall check whether this has actually not been done ."

print(translator_dict.get("you", "test"))

{'jag': 0.004204860403129542, 'fÃ¶rklarar': 1.6673198643673724e-19, 'europaparlamentets': 1.44994884455908e-13, 'session': 1.525047019471999e-23, 'Ã¥terupptagen': 5.632915285187233e-23, 'efter': 3.087336848316549e-10, 'avbrottet': 6.61304571750775e-23, 'den': 0.0002684332880868189, '17': 2.4299395771318723e-25, 'december': 9.307296215556794e-23, '.': 0.006965812624626439, 'vill': 0.00043648773805081263, 'pÃ¥': 0.0009584021016890107, 'nytt': 5.52380184129557e-10, 'Ã¶nska': 3.232833939368895e-16, 'er': 0.2510884626769683, 'ett': 8.737892715124028e-05, 'gott': 4.767541387323295e-16, 'Ã¥r': 8.311145744798248e-13, 'och': 0.0006103781174663259, 'hoppas': 7.871305193955057e-13, 'att': 0.0063030979040165315, 'ni': 0.6897485189976901, 'haft': 2.094980331487062e-13, 'en': 0.0003574979703661059, 'trevlig': 4.4665332501404345e-21, 'semester': 2.5729804812764544e-17, 'som': 0.0004402606009558609, 'kunnat': 1.2376333012597335e-23, 'konstatera': 1.8474276180271186e-19, 'Ã¤gde': 1.0383589123929229e-24

In [27]:
import evaluation

default = {"UNK": 0.0}

def translate(sentence):
    out = []
    for word in sentence.split(" "):
        out.append(max(translator_dict.get(word, default), key=lambda k: translator_dict.get(word, default)[k]))
    return " ".join(out)

evaluation.write_translatons_to_file("input_data.txt", "predictions_old.txt", translate)


100%|██████████| 1500/1500 [01:13<00:00, 20.41it/s]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f493d8cd-823d-4732-b316-78954073f53e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [28]:
evaluation.evaluate_model("reference.txt", "predictions_old.txt")


0.30643838364390297